In [1]:
from datasets import load_dataset
from dotenv import load_dotenv
from openai import OpenAI

from tqdm import tqdm

load_dotenv()

True

In [2]:
client = OpenAI()

paraphrasing_system_prompt = \
"""
You are a paraphrasing expert who is specialized in rewriting text (questions, statements, etc.) without altering the content. Keep in mind, that the meaning must not change after the paraphrasing. Just output the paraphrased text without any additional information.
"""

In [3]:
def paraphrase_input(input: str):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": paraphrasing_system_prompt},
            {"role": "user", "content": input}
        ],
        seed=42,
        temperature=1,
    )

    return response.choices[0].message.content

In [6]:
data = load_dataset("allenai/tulu-v2-sft-mixture", split="train")

Using the latest cached version of the dataset since allenai/tulu-v2-sft-mixture couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/lukashinterleitner/.cache/huggingface/datasets/allenai___tulu-v2-sft-mixture/default/0.0.0/6248b175d2ccb5ec7c4aeb22e6d8ee3b21b2c752 (last modified on Tue Jun 11 19:05:28 2024).


In [7]:
# select only data where dataset is "lima"
lima_data = data.filter(lambda x: x["dataset"] == "lima")

In [8]:
single_question_answers = lima_data.filter(lambda x: len(x["messages"]) == 2)
single_question_answers

Dataset({
    features: ['dataset', 'id', 'messages'],
    num_rows: 988
})

In [9]:
paraphrased = []

for row in tqdm(single_question_answers):
    paraphrased.append((row["id"], paraphrase_input(row["messages"][0]["content"])))

100%|██████████| 988/988 [18:17<00:00,  1.11s/it]


In [15]:
lima_data_paraphrased = single_question_answers.add_column("paraphrased_id", [p[0] for p in paraphrased])
lima_data_paraphrased = lima_data_paraphrased.add_column("paraphrased_messages", [[p[1],None] for p in paraphrased])

In [16]:
lima_data_paraphrased

Dataset({
    features: ['dataset', 'id', 'messages', 'paraphrased_id', 'paraphrased_messages'],
    num_rows: 988
})

In [17]:
test = True
for row in lima_data_paraphrased:
    test = test and (row["id"] == row["paraphrased_id"])
    
test

True

In [18]:
lima_data_paraphrased = lima_data_paraphrased.remove_columns("paraphrased_id")
lima_data_paraphrased

Dataset({
    features: ['dataset', 'id', 'messages', 'paraphrased_messages'],
    num_rows: 988
})

In [9]:
print(lima_data_paraphrased["messages"][0][0]["content"])

Can brain cells move? By movement I mean long distance migration (preferably within the brain only).


In [66]:
print(lima_data_paraphrased["messages"][0][1]["content"])

The question is relatively broad and one should take into account that the brain not only consists of neurons, but also glial cells (supportive cells) and pre-mitotic neuronal stem cells. Furthermore, as critical fellow-scientists have indicated, developmental stage is very important, as the developing embryonic brain is very different from the adult brain.
However, after sifting through various publications, the answer to the question is actually remarkably simple: Yes, brain cells migrate.
In  the adult brain glial cells migrate in the brain (Klämbt, 2009). Glial cells are involved in a myriad of functions, but a notable example of migrating glial cells are the oligodendrocytes that migrate relative long distances to find their target axons onto which they wrap themselves to form the insulating myelin sheath (Tsai and Miller, 2002).
Neuronal stem cells migrate over long distances in response to injury (Imitola et al., 2004) and they migrate from specific stem-cell locations (e.g., hi

In [67]:
print(lima_data_paraphrased["paraphrased_inputs"][0])

Is it possible for brain cells to migrate over long distances, specifically within the brain itself?


In [19]:
print(lima_data_paraphrased["messages"][1][0]["content"])

In our computer systems lecture we were introduced to the MIPS processor. It was (re)developed over the course of the term and has in fact been quite easy to understand. It uses a RISC design, that is its elementary commands are regularly encoded and there are only few of them in order to keep the wires simple.
It was mentioned that CISC follows a different philosophy. I looked briefly at the x86 instruction set and was shocked. I can not image how anyone would want to build a processor that uses so complex a command set!
So I figure there have to be good arguments why large portions of the processor market use CISC architectures. What are they? 


In [39]:
paraphrasing_system_prompt_2 = \
"""
You are a paraphrasing expert who is specialized in rewriting text (questions, statements, etc.). In your paraphrase, use the words MIPS, RISC and CISC.
"""

In [40]:
response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": paraphrasing_system_prompt_2},
            {"role": "user", "content": lima_data_paraphrased["messages"][0][0]["content"]}
        ],
        seed=42,
        temperature=1,
    )

response.choices[0].message.content

'Do neurons have the ability to migrate over long distances within the brain, in the context of discussions about instruction execution in computer processors, particularly comparing MIPS, RISC, and CISC architectures?'